# Bundle pose estimation by reflected rays
Find rigid pose (R, t) of the mirror bundle and assign observed reflected rays to mirrors.


Algorithm outline:
1. Load mirrors (local centers/normals) and observed rays (two points per ray).
2. Estimate ray normals: `n_est = normalize(d_in - d_out)` from observed directions.
3. Iterate: for current R, t build cost (angle + distance of mirror center to ray), find best mirror?ray assignment (rays < mirrors).
4. Update R with Kabsch on normals; update t to place centers onto closest points of assigned rays.
5. Stop when assignment/cost stabilizes.


In [7]:
import json
from dataclasses import dataclass
from pathlib import Path
from typing import List, Tuple
import itertools as it
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import least_squares
from scipy.spatial.transform import Rotation


In [8]:
BUNDLE_CONFIG = Path('bundle_config.txt')
RAYS_CONFIG = Path('rays_config.txt')
# Incoming ray: along -Z, hits the center of each mirror (laser from large Z to small Z)\n
D_IN = np.array([0.0, 0.0, -1.0])

In [9]:
@dataclass
class Mirror:
    name: str
    cut_deg: float
    tilt_deg: float
    height: float
    x: float
    y: float

@dataclass
class BundleConfig:
    fiber_diameter: float = 1.0
    pitch: float = 1.4
    label_offset: float = 0.6
    mirrors: List[Mirror] = None

def _parse_float(value: str, field: str, raw: str) -> float:
    try:
        return float(value)
    except ValueError as exc:
        raise ValueError(f'Failed to read {field} in line: {raw}') from exc

def load_bundle_config(path: Path) -> BundleConfig:
    mirrors: List[Mirror] = []
    globals_cfg = {}
    for raw in path.read_text(encoding='utf-8').splitlines():
        clean = raw.split('#', 1)[0].strip()
        if not clean:
            continue
        if clean.startswith('mirror'):
            parts = [p.strip() for p in clean.split(',') if p.strip()]
            params = {}
            for part in parts:
                if '=' in part:
                    k, v = part.split('=', 1)
                    params[k.strip()] = v.strip()
            name = params.get('mirror') or params.get('name')
            cut = _parse_float(params.get('cut', ''), 'cut', raw)
            tilt = _parse_float(params.get('tilt', '0'), 'tilt', raw)
            height = _parse_float(params.get('height', '0'), 'height', raw)
            x = _parse_float(params.get('x', '0'), 'x', raw)
            y = _parse_float(params.get('y', '0'), 'y', raw)
            mirrors.append(Mirror(name=name, cut_deg=cut, tilt_deg=tilt, height=height, x=x, y=y))
        else:
            if '=' not in clean:
                continue
            k, v = clean.split('=', 1)
            globals_cfg[k.strip()] = float(v.strip())
    return BundleConfig(
        fiber_diameter=float(globals_cfg.get('fiber_diameter', 1.0)),
        pitch=float(globals_cfg.get('pitch', globals_cfg.get('pitch_mm', 1.4))),
        label_offset=float(globals_cfg.get('label_offset', 0.6)),
        mirrors=mirrors,
    )

def load_rays_config(path: Path):
    rays = []
    for raw in path.read_text(encoding='utf-8').splitlines():
        clean = raw.split('#', 1)[0].strip()
        if not clean:
            continue
        parts = [p.strip() for p in clean.split(',') if p.strip()]
        p0 = p1 = None
        for p in parts:
            if p.startswith('p0='):
                vals = p.split('=', 1)[1].strip().split()
                p0 = np.array([float(v) for v in vals], dtype=float)
            if p.startswith('p1='):
                vals = p.split('=', 1)[1].strip().split()
                p1 = np.array([float(v) for v in vals], dtype=float)
        if p0 is None or p1 is None:
            raise ValueError(f'Missing p0/p1 in line: {raw}')
        rays.append((p0, p1))
    return rays

def mirror_geometry(m: Mirror):
    cut_rad = np.deg2rad(m.cut_deg)
    tilt_rad = np.deg2rad(m.tilt_deg)
    n_local = np.array([
        np.sin(cut_rad) * np.cos(tilt_rad),
        np.sin(cut_rad) * np.sin(tilt_rad),
        np.cos(cut_rad),
    ], dtype=float)
    c_local = np.array([m.x, m.y, m.height], dtype=float)
    # Orient normal so that its projection in XY points outward from the bundle center.
    if np.linalg.norm(c_local[:2]) > 1e-6:
        n_xy = n_local[:2]
        c_xy = c_local[:2]
        if np.dot(n_xy, c_xy) < 0:
            n_local = -n_local
    else:
        # For the central mirror (no XY radius), fall back to a convention
        # that the reflective side faces the incoming beam (dot(D_IN, n) < 0).
        if np.dot(D_IN, n_local) > 0:
            n_local = -n_local
    return n_local / np.linalg.norm(n_local), c_local


In [10]:
def solve_pose(bundle: BundleConfig, rays, *, w_ang=1.0, w_pos=1.0, w_div=0.0):
    """Jointly optimize R,t via least squares (angles + distances)."""
    normals_local = []
    centers_local = []
    for m in bundle.mirrors:
        n_loc, c_loc = mirror_geometry(m)
        normals_local.append(n_loc)
        centers_local.append(c_loc)
    normals_local = np.stack(normals_local)
    centers_local = np.stack(centers_local)

    d_obs = []
    ray_anchors = []
    for p0, p1 in rays:
        d = p1 - p0
        d = d / np.linalg.norm(d)
        d_obs.append(d)
        ray_anchors.append(p0)
    d_obs = np.stack(d_obs)
    ray_anchors = np.stack(ray_anchors)

    m = len(bundle.mirrors)
    k = len(rays)
    best = None

    mirrors_idx = list(range(m))
    rays_idx = list(range(k))

    for mirrors_subset in it.combinations(mirrors_idx, k):
        A = normals_local[list(mirrors_subset)]
        C = centers_local[list(mirrors_subset)]
        for perm in it.permutations(rays_idx):
            dirs = d_obs[list(perm)]
            anchors = ray_anchors[list(perm)]

            # Initial R via Kabsch on normals with sign alignment
            B = []
            for d in dirs:
                n_vec = D_IN - d
                n_vec = n_vec / np.linalg.norm(n_vec)
                B.append(n_vec)
            B = np.stack(B)
            for idx in range(len(B)):
                if np.dot(A[idx], B[idx]) < 0:
                    B[idx] = -B[idx]
            R0 = kabsch(A.T, B.T)

            # Initial t: least squares to lines (orthogonal projection)
            P = np.eye(3)[None, ...] - dirs[:, :, None] * dirs[:, None, :]
            Rc = (R0 @ C.T).T
            lhs = P.sum(axis=0)
            rhs = (P @ (anchors - Rc)[:, :, None]).sum(axis=0)[:, 0]
            try:
                t0 = np.linalg.solve(lhs, rhs)
            except np.linalg.LinAlgError:
                t0 = np.zeros(3, dtype=float)

            x0 = np.zeros(6, dtype=float)
            x0[:3] = Rotation.from_matrix(R0).as_rotvec()
            x0[3:] = t0

            def residuals(params):
                rvec = params[:3]
                t_vec = params[3:]
                Rm = Rotation.from_rotvec(rvec).as_matrix()
                n_w = (Rm @ A.T).T
                c_w = (Rm @ C.T).T + t_vec
                res = []
                for i in range(k):
                    n = n_w[i]
                    d_pred = D_IN - 2 * np.dot(n, D_IN) * n
                    d_pred = d_pred / np.linalg.norm(d_pred)
                    d_ray = dirs[i]
                    if np.dot(d_pred, d_ray) < 0:
                        d_ray = -d_ray
                    res_ang = w_ang * (d_pred - d_ray)
                    vec = c_w[i] - anchors[i]
                    proj = np.dot(vec, d_ray) * d_ray
                    dist_vec = vec - proj
                    res_dist = w_pos * dist_vec
                    if w_div > 0.0:
                        along = np.dot(vec, d_ray)
                        res_dist = res_dist + w_div * min(along, 0.0) * d_ray
                    res.append(res_ang)
                    res.append(res_dist)
                return np.concatenate(res)

            res = least_squares(residuals, x0, method='lm')
            cost = res.cost
            if best is None or cost < best[0]:
                R_best = Rotation.from_rotvec(res.x[:3]).as_matrix()
                t_best = res.x[3:]
                best = (cost, list(zip(mirrors_subset, perm)), R_best, t_best)

    if best is None:
        raise RuntimeError('Assignment not found')
    return best


In [11]:
def rotation_matrix_to_euler(R: np.ndarray):
    sy = np.sqrt(R[0,0]**2 + R[1,0]**2)
    singular = sy < 1e-8
    if not singular:
        x = np.arctan2(R[2,1], R[2,2])
        y = np.arctan2(-R[2,0], sy)
        z = np.arctan2(R[1,0], R[0,0])
    else:
        x = np.arctan2(-R[1,2], R[1,1])
        y = np.arctan2(-R[2,0], sy)
        z = 0.0
    return np.rad2deg(np.array([x, y, z]))  # roll, pitch, yaw (XYZ intrinsic)


In [12]:
AXIS_LOCAL = np.array([0.0, 0.0, 1.0])
COLORS = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']


def compute_world(bundle, R, t):
    normals = []
    centers = []
    for m in bundle.mirrors:
        n_loc, c_loc = mirror_geometry(m)
        normals.append(n_loc)
        centers.append(c_loc)
    normals = np.stack(normals)
    centers = np.stack(centers)
    normals_w = (R @ normals.T).T
    centers_w = (R @ centers.T).T + t
    return normals_w, centers_w


def ellipse_boundary(center, normal, axis_world, fiber_diameter, samples=64):
    minor_r = fiber_diameter * 0.5
    cos_theta = float(np.clip(np.dot(normal, axis_world), -1.0, 1.0))
    major_r = minor_r / max(abs(cos_theta), 1e-6)
    tangent = axis_world - cos_theta * normal
    if np.linalg.norm(tangent) < 1e-8:
        tangent = np.cross(normal, np.array([1.0, 0.0, 0.0]))
    if np.linalg.norm(tangent) < 1e-8:
        tangent = np.cross(normal, np.array([0.0, 1.0, 0.0]))
    e1 = tangent / np.linalg.norm(tangent)
    e2 = np.cross(normal, e1)
    theta = np.linspace(0.0, 2 * np.pi, samples)
    boundary = center + major_r * np.cos(theta)[:, None] * e1 + minor_r * np.sin(theta)[:, None] * e2
    return boundary


def add_slice(fig, center, size, plane, name):
    lin = np.linspace(-size, size, 6)
    if plane == 'xy':
        X, Y = np.meshgrid(lin + center[0], lin + center[1])
        Z = np.full_like(X, center[2])
    elif plane == 'xz':
        X, Z = np.meshgrid(lin + center[0], lin + center[2])
        Y = np.full_like(X, center[1])
    else:  # yz
        Y, Z = np.meshgrid(lin + center[1], lin + center[2])
        X = np.full_like(Y, center[0])
    fig.add_trace(go.Surface(x=X, y=Y, z=Z, showscale=False, opacity=0.08, colorscale=[[0, 'lightgray'], [1, 'lightgray']], name=name))


def clip_segment_to_box(p0, p1, bmin, bmax):
    p0 = p0.astype(float)
    p1 = p1.astype(float)
    d = p1 - p0
    t0, t1 = 0.0, 1.0
    for k in range(3):
        if abs(d[k]) < 1e-9:
            if p0[k] < bmin[k] or p0[k] > bmax[k]:
                return None
            continue
        inv = 1.0 / d[k]
        t_min = (bmin[k] - p0[k]) * inv
        t_max = (bmax[k] - p0[k]) * inv
        if t_min > t_max:
            t_min, t_max = t_max, t_min
        t0 = max(t0, t_min)
        t1 = min(t1, t_max)
        if t0 > t1:
            return None
    q0 = p0 + t0 * d
    q1 = p0 + t1 * d
    return q0, q1


def plot_solution(bundle, rays, R, t, match, ray_len=30.0, slice_scale=4.0):
    normals_w, centers_w = compute_world(bundle, R, t)
    axis_world = R @ AXIS_LOCAL
    fig = go.Figure()

    # Bounding box: 7 diameters centered on bundle center
    center_mean = centers_w.mean(axis=0)
    half_extent = bundle.fiber_diameter * 3.5
    bmin = center_mean - half_extent
    bmax = center_mean + half_extent

    add_slice(fig, center_mean, half_extent, 'xy', 'slice XY')
    add_slice(fig, center_mean, half_extent, 'xz', 'slice XZ')
    add_slice(fig, center_mean, half_extent, 'yz', 'slice YZ')

    # Compute reconstruction points (closest points on rays to mirror centers)
    hit_by_ray = {}
    for mi, rj in match:
        c = centers_w[mi]
        p0, p1 = rays[rj]
        d = p1 - p0
        d = d / np.linalg.norm(d)
        s = np.dot(d, c - p0)
        p_closest = p0 + s * d
        hit_by_ray[rj] = (p_closest, c)

    if hit_by_ray:
        hits = np.stack([hp for hp, _ in hit_by_ray.values()])
        recon_center = hits.mean(axis=0)
        towards_hits = recon_center - center_mean
    else:
        towards_hits = axis_world

    axis_draw = axis_world
    if np.dot(axis_draw, towards_hits) < 0:
        axis_draw = -axis_draw

    # Observed rays: draw only a clipped fragment around bundle center
    for j, (p0, p1) in enumerate(rays):
        d = p1 - p0
        d = d / np.linalg.norm(d)
        p_mid = p0 + d * np.dot(d, center_mean - p0)  # closest point to bundle center
        seg_start = p_mid - d * half_extent
        seg_end = p_mid + d * half_extent
        clipped = clip_segment_to_box(seg_start, seg_end, bmin, bmax)
        if clipped is None:
            continue
        q0, q1 = clipped
        fig.add_trace(go.Scatter3d(
            x=[q0[0], q1[0]],
            y=[q0[1], q1[1]],
            z=[q0[2], q1[2]],
            mode='lines',
            line=dict(color='rgba(0,0,150,0.7)', width=5),
            name=f'obs ray {j}'
        ))

    # Segments from reconstruction points to mirror centers (residuals), clipped
    for j, (p_hit, c) in hit_by_ray.items():
        clipped = clip_segment_to_box(p_hit, c, bmin, bmax)
        if clipped is None:
            continue
        q0, q1 = clipped
        fig.add_trace(go.Scatter3d(
            x=[q0[0], q1[0]],
            y=[q0[1], q1[1]],
            z=[q0[2], q1[2]],
            mode='lines',
            line=dict(color='rgba(50,50,50,0.5)', width=2, dash='dot'),
            showlegend=False,
        ))

    # Predicted rays and mirror apertures, clipped to the bounding box
    for i, (c, n) in enumerate(zip(centers_w, normals_w)):
        color = COLORS[i % len(COLORS)]
        # incoming ray to center
        inc_start = c - D_IN * half_extent
        clipped_inc = clip_segment_to_box(inc_start, c, bmin, bmax)
        if clipped_inc is not None:
            q0, q1 = clipped_inc
            fig.add_trace(go.Scatter3d(
                x=[q0[0], q1[0]],
                y=[q0[1], q1[1]],
                z=[q0[2], q1[2]],
                mode='lines',
                line=dict(color='rgba(100,100,100,0.7)', width=3, dash='dash'),
                name=f'incoming to M{i}' if i == 0 else None,
                showlegend=(i == 0),
            ))
        # reflected
        d_pred = D_IN - 2 * np.dot(n, D_IN) * n
        d_pred = d_pred / np.linalg.norm(d_pred)
        end = c + d_pred * half_extent
        clipped_ref = clip_segment_to_box(c, end, bmin, bmax)
        if clipped_ref is not None:
            q0, q1 = clipped_ref
            fig.add_trace(go.Scatter3d(
                x=[q0[0], q1[0]],
                y=[q0[1], q1[1]],
                z=[q0[2], q1[2]],
                mode='lines',
                line=dict(color=color, width=4),
                name=f'mirror {i} pred',
            ))
        boundary = ellipse_boundary(c, n, axis_draw, bundle.fiber_diameter)
        fig.add_trace(go.Scatter3d(
            x=boundary[:, 0],
            y=boundary[:, 1],
            z=boundary[:, 2],
            mode='lines',
            line=dict(color=color, width=2),
            showlegend=False,
        ))
        fig.add_trace(go.Scatter3d(
            x=[c[0]],
            y=[c[1]],
            z=[c[2]],
            mode='markers+text',
            marker=dict(color=color, size=4),
            text=[f'M{i}'],
            textposition='top center',
            showlegend=False,
        ))

    fig.update_layout(
        height=900,
        width=900,
        title='Observed rays vs predicted reflected rays',
        scene=dict(
            aspectmode='cube',
            aspectratio=dict(x=1, y=1, z=1),
            xaxis=dict(title='X', range=[bmin[0], bmax[0]]),
            yaxis=dict(title='Y', range=[bmin[1], bmax[1]]),
            zaxis=dict(title='Z', range=[bmin[2], bmax[2]]),
        ),
        scene_camera=dict(projection=dict(type='orthographic')),
        margin=dict(l=0, r=0, t=40, b=0),
    )
    fig.show()


In [13]:
bundle = load_bundle_config(BUNDLE_CONFIG)
rays = load_rays_config(RAYS_CONFIG)
total, match, R, t = solve_pose(bundle, rays, w_ang=1.0, w_pos=1.0, w_div=10.0)
euler = rotation_matrix_to_euler(R)
print(f'Total cost: {total:.4f}')
print('Rotation R:')
print(R)
print(f'Euler XYZ (deg): roll={euler[0]:.3f}, pitch={euler[1]:.3f}, yaw={euler[2]:.3f}')
print('Translation t:')
print(t)
print('Assignment (mirror idx -> ray idx):', match)
plot_solution(bundle, rays, R, t, match, ray_len=30.0)


NameError: name 'kabsch' is not defined

In [ ]:
# Synthetic self-test
def generate_rays_from_bundle(bundle, R_true, t_true, ray_len=30.0):
    normals = []
    centers = []
    for m in bundle.mirrors:
        n_loc, c_loc = mirror_geometry(m)
        normals.append(n_loc)
        centers.append(c_loc)
    normals = np.stack(normals)
    centers = np.stack(centers)
    normals_w = (R_true @ normals.T).T
    centers_w = (R_true @ centers.T).T + t_true
    rays = []
    for c, n in zip(centers_w, normals_w):
        d_out = D_IN - 2 * np.dot(n, D_IN) * n
        d_out = d_out / np.linalg.norm(d_out)
        if np.dot(d_out, n) < 0:
            d_out = -d_out
        p0 = c.copy()
        p1 = c + d_out * ray_len
        rays.append((p0, p1))
    return rays

theta = np.deg2rad(10.0)
R_true = np.array([[np.cos(theta), -np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0], [0, 0, 1]])
t_true = np.array([5.0, -3.0, 10.0])
synthetic_rays = generate_rays_from_bundle(bundle, R_true, t_true, ray_len=30.0)
total_s, match_s, R_hat, t_hat = solve_pose(bundle, synthetic_rays, w_ang=1.0, w_pos=1.0, w_div=10.0)
rot_err = np.linalg.norm(R_true - R_hat)
trans_err = np.linalg.norm(t_true - t_hat)
print('Synthetic test:')
print(f'  Total cost: {total_s:.4f}')
print(f'  Rotation error Frobenius: {rot_err:.6e}')
print(f'  Translation error norm:   {trans_err:.6e}')
euler_hat = rotation_matrix_to_euler(R_hat)
print(f'  Euler XYZ (deg) est: roll={euler_hat[0]:.3f}, pitch={euler_hat[1]:.3f}, yaw={euler_hat[2]:.3f}')
print('  Assignment (mirror idx -> ray idx):', match_s)
plot_solution(bundle, synthetic_rays, R_hat, t_hat, match_s, ray_len=30.0)


Synthetic test:
  Total cost: 2.1186
  Rotation error Frobenius: 2.828427e+00
  Translation error norm:   2.702123e+00
  Euler XYZ (deg) est: roll=91.053, pitch=-0.000, yaw=-170.000
  Assignment (mirror idx -> ray idx): [(0, 3), (1, 1), (2, 2), (3, 0), (4, 5), (5, 4), (6, 6)]


In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment
import math

# 1. Вспомогательные функции

def normalize(v, axis=-1, eps=1e-12):
    v = np.asarray(v, float)
    n = np.linalg.norm(v, axis=axis, keepdims=True)
    n = np.maximum(n, eps)
    return v / n

def kabsch(A, B):
    A = np.asarray(A, float)
    B = np.asarray(B, float)
    H = A.T @ B
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T
    return R

def reflect_dir(d_in, n):
    d_in = normalize(d_in, axis=-1)
    n = normalize(n, axis=-1)
    dot = np.sum(d_in * n, axis=-1, keepdims=True)
    return d_in - 2.0 * dot * n

def normal_from_dirs(d_in, d_out):
    d_in = normalize(d_in, axis=-1)
    d_out = normalize(d_out, axis=-1)
    n = normalize(d_out - d_in, axis=-1)
    dot = np.sum(n * d_in, axis=-1, keepdims=True)
    mask = dot >= 0.0
    n = np.where(mask, -n, n)
    return n

def random_rotation(rng):
    axis = normalize(rng.normal(size=3))
    angle = rng.uniform(0, 2*np.pi)
    ux, uy, uz = axis
    c = math.cos(angle)
    s = math.sin(angle)
    C = 1.0 - c
    R = np.array([
        [c + ux*ux*C,     ux*uy*C - uz*s, ux*uz*C + uy*s],
        [uy*ux*C + uz*s,  c + uy*uy*C,    uy*uz*C - ux*s],
        [uz*ux*C - uy*s,  uz*uy*C + ux*s, c + uz*uz*C   ],
    ])
    return R

def small_random_rotation(rng, max_deg=5.0):
    axis = normalize(rng.normal(size=3))
    angle = rng.uniform(-max_deg, max_deg) * np.pi / 180.0
    ux, uy, uz = axis
    c = math.cos(angle)
    s = math.sin(angle)
    C = 1.0 - c
    R = np.array([
        [c + ux*ux*C,     ux*uy*C - uz*s, ux*uz*C + uy*s],
        [uy*ux*C + uz*s,  c + uy*uy*C,    uy*uz*C - ux*s],
        [uz*ux*C - uy*s,  uz*uy*C + ux*s, c + uz*uz*C   ],
    ])
    return R

def distance_point_to_ray_sq(point, ray_p0, ray_dir):
    d = normalize(ray_dir)
    diff = point - ray_p0
    proj = diff - np.dot(diff, d) * d
    return np.dot(proj, proj)

# 2. Простая геометрия тестовой сборки

def make_test_bundle(nx=3, ny=3, pitch=1.0, sagitta=-3.0, D_in=np.array([0., 0., -1.])):
    xs = (np.arange(nx) - (nx - 1) / 2.0) * pitch
    ys = (np.arange(ny) - (ny - 1) / 2.0) * pitch
    centers_local = []
    normals_local = []
    target = np.array([0., 0., sagitta])
    for y in ys:
        for x in xs:
            c = np.array([x, y, 0.0])
            centers_local.append(c)
            n = normalize(target - c)
            if np.dot(n, D_in) > 0.0:
                n = -n
            normals_local.append(n)
    return np.asarray(centers_local), np.asarray(normals_local)

# 3. Симуляция отражений

def apply_pose(points, R, t):
    return (R @ points.T).T + t

def simulate_rays(centers_local, normals_local, R, t, D_in):
    centers_world = apply_pose(centers_local, R, t)
    normals_world = (R @ normals_local.T).T
    d_in = np.broadcast_to(D_in, centers_world.shape)
    d_out = reflect_dir(d_in, normals_world)
    p0 = centers_world + 0.5 * d_out
    p1 = p0 + d_out
    return p0, p1, centers_world, normals_world, d_out

# 4. Восстановление позы при известном биндинге

def estimate_pose_known_assignment(p0, p1, centers_local, normals_local, assignment, D_in):
    p0 = np.asarray(p0, float)
    p1 = np.asarray(p1, float)
    centers_local = np.asarray(centers_local, float)
    normals_local = np.asarray(normals_local, float)
    assignment = np.asarray(assignment, int)
    N = p0.shape[0]
    d_obs = normalize(p1 - p0, axis=1)
    n_est = normal_from_dirs(D_in, d_obs)
    A_local = normals_local[assignment]
    B_world = n_est
    R = kabsch(A_local, B_world)
    I = np.eye(3)
    A_rows = []
    b_rows = []
    for i in range(N):
        j = assignment[i]
        c_rot = R @ centers_local[j]
        d = d_obs[i]
        P = I - np.outer(d, d)
        A_rows.append(P)
        b_rows.append(P @ (p0[i] - c_rot))
    A = np.concatenate(A_rows, axis=0)
    b = np.concatenate(b_rows, axis=0)
    t, *_ = np.linalg.lstsq(A, b, rcond=None)
    return R, t

# 5. Восстановление позы при неизвестном биндинге

def estimate_pose_unknown_assignment(p0, p1, centers_local, normals_local, D_in,
    w_normals=1.0, w_centers=0.1, max_iter=50, tol_angle_deg=1e-3, verbose=False):
    p0 = np.asarray(p0, float)
    p1 = np.asarray(p1, float)
    centers_local = np.asarray(centers_local, float)
    normals_local = np.asarray(normals_local, float)
    N = p0.shape[0]
    M = centers_local.shape[0]
    assert N <= M
    d_obs = normalize(p1 - p0, axis=1)
    n_est = normal_from_dirs(D_in, d_obs)
    R = np.eye(3)
    assignment = np.arange(N)
    I = np.eye(3)
    for it in range(max_iter):
        A_rows = []
        b_rows = []
        for i in range(N):
            j = assignment[i]
            c_rot = R @ centers_local[j]
            d = d_obs[i]
            P = I - np.outer(d, d)
            A_rows.append(P)
            b_rows.append(P @ (p0[i] - c_rot))
        A = np.concatenate(A_rows, axis=0)
        b = np.concatenate(b_rows, axis=0)
        t, *_ = np.linalg.lstsq(A, b, rcond=None)
        centers_pred = apply_pose(centers_local, R, t)
        normals_pred = (R @ normals_local.T).T
        dot = n_est @ normals_pred.T
        normal_cost = 1.0 - dot
        center_cost = np.zeros((N, M))
        for i in range(N):
            for j in range(M):
                center_cost[i, j] = distance_point_to_ray_sq(centers_pred[j], p0[i], d_obs[i])
        cost = w_normals * normal_cost + w_centers * center_cost
        row_ind, col_ind = linear_sum_assignment(cost)
        assert np.all(row_ind == np.arange(N))
        new_assignment = col_ind
        A_local = normals_local[new_assignment]
        B_world = n_est
        R_new = kabsch(A_local, B_world)
        R_delta = R_new @ R.T
        tr = np.trace(R_delta)
        tr = np.clip(tr, -1.0, 3.0)
        angle = math.degrees(math.acos((tr - 1.0) / 2.0))
        changed = np.any(new_assignment != assignment)
        if verbose:
            print(f"iter {it:02d}: angle_step={angle:.6f}°, changed={changed}")
        R = R_new
        assignment = new_assignment
        if angle < tol_angle_deg and not changed:
            break
    A_rows = []
    b_rows = []
    for i in range(N):
        j = assignment[i]
        c_rot = R @ centers_local[j]
        d = d_obs[i]
        P = I - np.outer(d, d)
        A_rows.append(P)
        b_rows.append(P @ (p0[i] - c_rot))
    A = np.concatenate(A_rows, axis=0)
    b = np.concatenate(b_rows, axis=0)
    t, *_ = np.linalg.lstsq(A, b, rcond=None)
    return R, t, assignment
